In [177]:
'''
Steps for Data Cleaning:
>punctiation, numbers, case, etc.
>manufacturer references
>device references
>"EVENT DESCRIPTION"
'''

'\nSteps for Data Cleaning:\n>punctiation, numbers, case, etc.\n>manufacturer references\n>device references\n>"EVENT DESCRIPTION"\n'

In [398]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

csv.field_size_limit(1000000000000000000)

1000000000000000000

In [399]:
# import FOITEXT and MASDEV files
left = pd.read_pickle("./3_FOITEXT_File.pkl")
right = pd.read_pickle("./3_MASTER_DEVICE_File_wCENSOR_NOTCENSORED")

In [400]:
print("shape of FOITEXT: " + str(left.shape))
print("shape of MASTER_DEVICE: " + str(right.shape))

shape of FOITEXT: (10419137, 7)
shape of MASTER_DEVICE: (49963, 34)


In [401]:
df = pd.merge(left, right, on="MDR_REPORT_KEY", how="inner", indicator=True)
df["_merge"].value_counts()

both          42208
left_only         0
right_only        0
Name: _merge, dtype: int64

In [402]:
# 7755 files from MASDEV did not have an associating MDR_REPORT_KEY

In [403]:
df.head()

,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT,textLen,REPORT_NUMBER,REPORT_SOURCE_CODE,MANUFACTURER_LINK_FLAG_,...,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,year,STD_MANUFACTURER,STD_DEVICE,RECALL_CLASS,Before2013,CENSORED,_merge
0,262717,162068,D,1,NaN,EVENT DESCRIPTION CPI RECEIVED INFORMATION THA...,NaN,2124215-2000-01321,M,Y,...,LWS,0 DAY,R,2000,BOSTON SCIENTIFIC,BOS_SCI_ENDOTAK,0,True,2000-02-09,both
1,262790,161157,D,1,NaN,EVENT DESCRIPTION CPI RECEIVED INFORMATION THA...,NaN,2124215-2000-01260,M,Y,...,LWS,4 YR,Y,2000,BOSTON SCIENTIFIC,BOS_SCI_ENDOTAK_DSP,0,True,2000-02-09,both
2,262799,204642,D,1,NaN,EVENT DESCRIPTION GUIDANT/CRM RECEIVED INFORMA...,NaN,2124215-2000-01516,M,Y,...,LWS,0 DAY,R,2000,BOSTON SCIENTIFIC,BOS_SCI_SELUTE,0,True,2000-02-09,both
3,262812,175882,D,1,NaN,EVENT DESCRIPTION GUIDANT/CPI RECEIVED INFORMA...,NaN,2124215-2000-01503,M,Y,...,LWS,29 MO,R,2000,BOSTON SCIENTIFIC,BOS_SCI_SELUTE,0,True,2000-02-09,both
4,262819,175883,D,1,NaN,EVENT DESCRIPTION CPI RECEIVED INFORMATION THA...,NaN,2124215-2000-01510,M,Y,...,LWS,5 YR,Y,2000,BOSTON SCIENTIFIC,BOS_SCI_ENDOTAK,0,True,2000-02-09,both


In [404]:
# make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.
def cleanTextRound1(text):
    text = text.upper()
    text = re.sub('\[.*?\‘’“”…]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    return text

round1 = lambda x: cleanTextRound1(x)


data_clean = pd.DataFrame(df.FOI_TEXT.apply(round1))
data_clean.head(5)

,FOI_TEXT
0,EVENT DESCRIPTION CPI RECEIVED INFORMATION THA...
1,EVENT DESCRIPTION CPI RECEIVED INFORMATION THA...
2,EVENT DESCRIPTION GUIDANTCRM RECEIVED INFORMAT...
3,EVENT DESCRIPTION GUIDANTCPI RECEIVED INFORMAT...
4,EVENT DESCRIPTION CPI RECEIVED INFORMATION THA...


In [405]:
# checking the first 100 reports in the df
n=0

for entry in data_clean["FOI_TEXT"]:
    if n < 100:
        print("[" + str(n) + "]\n" + entry + "\n")
        n += 1    

[0]
EVENT DESCRIPTION CPI RECEIVED INFORMATION THAT THIS IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD IDENTIFIED A  OHM SHOCKING IMPEDANCE DURING AN IMPLANT PROCEDURE  CPIS TECHNICAL SERVICES RECOMMENDED THE DEVICE NOT BE IMPLANTED AND THAT FURTHER LEAD TESTING BE PERFORMED

[1]
EVENT DESCRIPTION CPI RECEIVED INFORMATION THAT THIS PATIENT WITH AN ENDOTAK TRANSVENOUS DEFIBRILLATION LEAD DIED DUE TO NON TERMINATION OF VF THE SHOCKING IMPEDANCE REPORTED WAS 

[2]
EVENT DESCRIPTION GUIDANTCRM RECEIVED INFORMATION THAT THIS IMPLANTABLE LEAD WAS FLOATING IN THE HEART

[3]
EVENT DESCRIPTION GUIDANTCPI RECEIVED INFORMATION THAT DUE TO MICROPERFORATION AND MICRODISLODGMENT WITH A RISE IN THE THRESHOLD THIS SELUTE PICOTIP LEAD WAS REMOVED FROM SERVICE AND REPLACED WITH ANOTHER GUIDANT LEAD

[4]
EVENT DESCRIPTION CPI RECEIVED INFORMATION THAT THIS ENDOTAK TRANSVENOUS DEFIBRILLATION LEAD WAS REMOVED FROM SERVICE  THE PATIENT CAME INTO THE EMERGENCY ROOM AFTER EXPERIENCING A SHOCK  THE IMPANTABLE CAR

In [406]:
# indexes 2, 3, 13, 14, 50 were selected to see if cleanTextRound2 worked
for n in [2,3,13,14,50]:
    print("[" + str(n) + "]\n" + data_clean["FOI_TEXT"][n] + "\n")
    
# indexes 4, 8, 13, 14, 70 were selected to see if cleanTextRound3 worked
for n in [4, 8, 13, 14, 70]:
    print("[" + str(n) + "]\n" + data_clean["FOI_TEXT"][n] + "\n")
    
# indexes 8, 9, 10, 20, 21 were selected to see if cleanTextRound5 worked
for n in [8, 9, 10, 20, 21]:
    print("[" + str(n) + "]\n" + data_clean["FOI_TEXT"][n] + "\n")

[2]
EVENT DESCRIPTION GUIDANTCRM RECEIVED INFORMATION THAT THIS IMPLANTABLE LEAD WAS FLOATING IN THE HEART

[3]
EVENT DESCRIPTION GUIDANTCPI RECEIVED INFORMATION THAT DUE TO MICROPERFORATION AND MICRODISLODGMENT WITH A RISE IN THE THRESHOLD THIS SELUTE PICOTIP LEAD WAS REMOVED FROM SERVICE AND REPLACED WITH ANOTHER GUIDANT LEAD

[13]
EVENT DESCRIPTION CPI RECEIVED INFORMATION THAT THIS ENDOTAK DSP TRANSVENOUS DEFIBRILLATION LEAD WAS OVERSENSING AND DELIVERED AN INAPPROPRIATE SHOCK OVERSENSING COULD BE REPRODUCED WITH ARM MOVEMENT DURING LEAD REVISION DAMAGE WAS NOTED AT THE JUNCTURE OF  TERMINAL PIN

[14]
EVENT DESCRIPTION CPI RECEIVED INFORMATION THAT THIS IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD SYSTEM WAS UNDERSENSING EPISODES OF VT AND VF THE RATE SENSE TERMINAL OF THE ENDOTAK LEAD WAS CAPPED AND A NEW RATE SENSE LEAD WAS ADDED TO THE SYSTEM  DURING THE PROCEDURE THE PHYSICIAN ELECTED TO REMOVE THE ICD AS WELL

[50]
EVENT DESCRIPTION GUIDANT RECEIVED INFORMATION THAT THE PACESENS

In [407]:
# replace manufacturer identifiers
# spaces were not used in regexs as words may not be spaced
def cleanTextRound2(text):
    text = re.sub('''SJM|TELECTRONICS PACING SYSTEMS INC|RICE CREEK|MPROC|MDT|EOC HEERLEN DISTRIBUTION CTR
                        |BOSTON SCIENTIFIC|GUIDANT|CPI|CARDIAC PACEMAKER|CLONMEL|CAMERON HEALTH|HISTORICAL PUERTO RICO
                        |EXTERNAL MANUFACTURERPCI PLANT|SORI|ANGEION|MPRI|IPG MFG|ST JUDE|ST JUDE MEDICAL|MEDTRONIC 
                        |BOSTON SCIENTIFIC|BIOTRONIK|SORIN''', " MFGID ", text)
    return text

round2 = lambda x: cleanTextRound2(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round2))
data_clean.head(5)

,FOI_TEXT
0,EVENT DESCRIPTION MFGID RECEIVED INFORMATION...
1,EVENT DESCRIPTION MFGID RECEIVED INFORMATION...
2,EVENT DESCRIPTION MFGID CRM RECEIVED INFORMAT...
3,EVENT DESCRIPTION MFGID MFGID RECEIVED INFO...
4,EVENT DESCRIPTION MFGID RECEIVED INFORMATION...


In [408]:
# indexes 2, 3, 13, 14, 50 were selected to see if cleanTextRound2 worked
for n in [2,3,13,14,50]:
    print("[" + str(n) + "]\n" + data_clean["FOI_TEXT"][n] + "\n")
# it was successful

[2]
EVENT DESCRIPTION  MFGID CRM RECEIVED INFORMATION THAT THIS IMPLANTABLE LEAD WAS FLOATING IN THE HEART

[3]
EVENT DESCRIPTION  MFGID  MFGID  RECEIVED INFORMATION THAT DUE TO MICROPERFORATION AND MICRODISLODGMENT WITH A RISE IN THE THRESHOLD THIS SELUTE PICOTIP LEAD WAS REMOVED FROM SERVICE AND REPLACED WITH ANOTHER  MFGID  LEAD

[13]
EVENT DESCRIPTION  MFGID  RECEIVED INFORMATION THAT THIS ENDOTAK DSP TRANSVENOUS DEFIBRILLATION LEAD WAS OVERSENSING AND DELIVERED AN INAPPROPRIATE SHOCK OVERSENSING COULD BE REPRODUCED WITH ARM MOVEMENT DURING LEAD REVISION DAMAGE WAS NOTED AT THE JUNCTURE OF  TERMINAL PIN

[14]
EVENT DESCRIPTION  MFGID  RECEIVED INFORMATION THAT THIS IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD SYSTEM WAS UNDERSENSING EPISODES OF VT AND VF THE RATE SENSE TERMINAL OF THE ENDOTAK LEAD WAS CAPPED AND A NEW RATE SENSE LEAD WAS ADDED TO THE SYSTEM  DURING THE PROCEDURE THE PHYSICIAN ELECTED TO REMOVE THE ICD AS WELL

[50]
EVENT DESCRIPTION  MFGID  RECEIVED INFORMATION THAT 

In [409]:
# creating a list of device identifiers
listDevices = []



for n in df["STD_DEVICE"].value_counts().index:
    listDevices.append(n)

print(listDevices)

['BOS_SCI_ENDOTAK_RELIANCE', 'STJUDE_DURATA', 'STJUDE_RIATA', 'MEDTRONIC_SPRINT_QUATTRO_SECURE', 'MEDTRONIC_SPRINT_QUATTRO', 'MEDTRONIC_SPRINT', 'BIOTRONIK_LINOX', 'MEDTRONIC_TRANSVENE', 'STJUDE_RIATA_ST', 'MEDTRONIC_SPRINT_FIDELIS', 'BIOTRONIK_LUMAX', 'STJUDE_RIATA_ST_OPTIM', 'STJUDE_QUICKFLEX', 'BOS_SCI_ENDOTAK_ENDURANCE', 'BOS_SCI_ENDOTAK_DSP', 'BIOTRONIK_LINOX_SMART', 'BOS_SCI_ENDOTAK', 'SORIN_ISOLINE', 'STJUDE_TVL_ADX', 'BOS_SCI_EASYTRAK', 'STJUDE_QUICKSITE', 'BOS_SCI_SELUTE', 'BOS_SCI_EASYTRAK2', 'STJUDE_SPL', 'BIOTRONIK_KENTROX_STEROID', 'MEDTRONIC_SUBQ', 'MEDTRONIC_CAPSURE', 'BIOTRONIK_KENTROX', 'BIOTRONIK_LINOX_SMART_PROMRI', 'MEDTRONIC_EPICARDIAL_PATCH', 'STJUDE_TVL', 'MEDTRONIC_SELECTSECURE', 'BIOTRONIK_COROX_OTW', 'BOS_SCI_INTERVENE', 'BIOTRONIK_KAINOX', 'BOS_SCI_ENDOTAK_SQ', 'BOS_SCI_EASYTRAK3', 'BIOTRONIK_SELOX', 'MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI', 'BOS_SCI_FLEXTEND2', 'BIOTRONIK_VIGILA_1', 'SORIN_SITUS', 'MEDTRONIC_ATTAIN', 'BIOTRONIK_VIGILA_2', 'BOS_SCI_ACUITY', 'BIO

In [410]:
# replace device identifiers
# other versions of existing entries were added to account for typos
# spaces were not used in regexs as words may not be spaced
def cleanTextRound3(text):
    text = re.sub('''
                    ENDOTAK|ENDOTAK RELIANCE|DURATA|RIATA|SPRINT QUATTRO SECURE|SPRINT QUATTRO|SPRINT|LINOX|TRANSVENE
                    |RIATA ST|SPRINT FIDELIS|LUMAX|RIATA ST OPTIM|QUICKFLEX|ENDOTAK ENDURANCE|ENDOTAK DSP|LINOX SMART 
                    |ENDOTAK|ISOLINE|TVL ADX|EASYTRAK|QUICKSITE|SELUTE|EASYTRAK 2|EASYTRAK2|SPL|KENTROX STEROID|SUBQ
                    |CAPSURE|KENTROX|LINOX SMART PROMRI|EPICARDIAL PATCH|TVL|SELECTSECURE|SELECT SECURE
                    |COROX OTW|COROX|INTERVENE|KAINOX|ENDOTAK SQ|EASYTRAK3|BIOTRONIK_SELOX|SPRINT QUATTRO SECURE MRI
                    |FLEXTEND 2|FEXTEND2|VIGILA 1|VIGILA1|SORIN_SITUS| MEDTRONIC_ATTAIN|VIGILA2|VIGILA 1
                    |ACUITY|AROX|QTRAK|STELID II|STELIDII|STELID2|STELID 2|FINELINE|TENDRIL|FLEXTEND|STELIX I|STELIXII
                    |STELIX 2|STELIX2|
                    ''', " DEVID ", text)
    return text

round3 = lambda x: cleanTextRound3(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round3))
data_clean.head(5)

,FOI_TEXT
0,EVENT DESCRIPTION MFGID RECEIVED INFORMATION...
1,EVENT DESCRIPTION MFGID RECEIVED INFORMATION...
2,EVENT DESCRIPTION MFGID CRM RECEIVED INFORMAT...
3,EVENT DESCRIPTION MFGID MFGID RECEIVED INFO...
4,EVENT DESCRIPTION MFGID RECEIVED INFORMATION...


In [411]:
# indexes 4, 8, 13, 14, 70  were selected to see if cleanTextRound3 worked
for n in [4, 8, 13, 14, 70]:
    print("[" + str(n) + "]\n" + data_clean["FOI_TEXT"][n] + "\n")
# it was sucessful

[4]
EVENT DESCRIPTION  MFGID  RECEIVED INFORMATION THAT THIS  DEVID  TRANSVENOUS DEFIBRILLATION LEAD WAS REMOVED FROM SERVICE  THE PATIENT CAME INTO THE EMERGENCY ROOM AFTER EXPERIENCING A SHOCK  THE IMPANTABLE CARDIOVERTER DEFIBRILLATOR ICD WAS INTERROGATED AND DI DEVID AYED A LESS THAN  IMPEDANCE READING  THE ICD WAS ELECTIVELY REMOVED AT THE SAME PROCEDURE

[8]
EVENT DESCRIPTION  MFGID  RECEIVED INFORMATION THAT THIS  DEVID  TRANSVENOUS DEFIBRILLATION LEAD EXHIBITED LESS THAN  OHMS OF SHOCKING IMPEDANCE THE LEAD AND IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD WERE REMOVED FROM SERVICE

[13]
EVENT DESCRIPTION  MFGID  RECEIVED INFORMATION THAT THIS  DEVID  TRANSVENOUS DEFIBRILLATION LEAD WAS OVERSENSING AND DELIVERED AN INAPPROPRIATE SHOCK OVERSENSING COULD BE REPRODUCED WITH ARM MOVEMENT DURING LEAD REVISION DAMAGE WAS NOTED AT THE JUNCTURE OF  TERMINAL PIN

[14]
EVENT DESCRIPTION  MFGID  RECEIVED INFORMATION THAT THIS IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD SYSTEM WAS UNDERSENSING

In [412]:
# replace "EVENT DESCRIPTION" found at the start of many entries
def cleanTextRound4(text):
    text = re.sub("EVENT DESCRIPTION", "", text)
    return text

round4 = lambda x: cleanTextRound4(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round4))
data_clean.head(5)

,FOI_TEXT
0,MFGID RECEIVED INFORMATION THAT THIS IMPLAN...
1,MFGID RECEIVED INFORMATION THAT THIS PATIEN...
2,MFGID CRM RECEIVED INFORMATION THAT THIS IMP...
3,MFGID MFGID RECEIVED INFORMATION THAT DUE ...
4,MFGID RECEIVED INFORMATION THAT THIS DEVID...


In [413]:
# indexes 4, 8, 13, 14, 70  were selected to see if cleanTextRound4 worked
for n in [4, 8, 13, 14, 70]:
    print("[" + str(n) + "]\n" + data_clean["FOI_TEXT"][n] + "\n")
# it was sucessful

[4]
  MFGID  RECEIVED INFORMATION THAT THIS  DEVID  TRANSVENOUS DEFIBRILLATION LEAD WAS REMOVED FROM SERVICE  THE PATIENT CAME INTO THE EMERGENCY ROOM AFTER EXPERIENCING A SHOCK  THE IMPANTABLE CARDIOVERTER DEFIBRILLATOR ICD WAS INTERROGATED AND DI DEVID AYED A LESS THAN  IMPEDANCE READING  THE ICD WAS ELECTIVELY REMOVED AT THE SAME PROCEDURE

[8]
  MFGID  RECEIVED INFORMATION THAT THIS  DEVID  TRANSVENOUS DEFIBRILLATION LEAD EXHIBITED LESS THAN  OHMS OF SHOCKING IMPEDANCE THE LEAD AND IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD WERE REMOVED FROM SERVICE

[13]
  MFGID  RECEIVED INFORMATION THAT THIS  DEVID  TRANSVENOUS DEFIBRILLATION LEAD WAS OVERSENSING AND DELIVERED AN INAPPROPRIATE SHOCK OVERSENSING COULD BE REPRODUCED WITH ARM MOVEMENT DURING LEAD REVISION DAMAGE WAS NOTED AT THE JUNCTURE OF  TERMINAL PIN

[14]
  MFGID  RECEIVED INFORMATION THAT THIS IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD SYSTEM WAS UNDERSENSING EPISODES OF VT AND VF THE RATE SENSE TERMINAL OF THE  DEVID  LEAD W

In [414]:
# replace odd spacing
def cleanTextRound5(text):
    text = re.sub(" ", " ", text)
    text = re.sub("  ", " ", text)
    text = re.sub("   ", " ", text)
    text = re.sub("    ", " ", text)
    text = re.sub("     ", " ", text)
    return text

round5 = lambda x: cleanTextRound5(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round5))
data_clean.head(5)

,FOI_TEXT
0,MFGID RECEIVED INFORMATION THAT THIS IMPLANTA...
1,MFGID RECEIVED INFORMATION THAT THIS PATIENT ...
2,MFGID CRM RECEIVED INFORMATION THAT THIS IMPL...
3,MFGID MFGID RECEIVED INFORMATION THAT DUE TO ...
4,MFGID RECEIVED INFORMATION THAT THIS DEVID TR...


In [415]:
# indexes 8, 9, 10, 20, 21 were selected to see if cleanTextRound5 worked
for n in [8, 9, 10, 20, 21]:
    print("[" + str(n) + "]\n" + data_clean["FOI_TEXT"][n] + "\n")
# it was successful

[8]
 MFGID RECEIVED INFORMATION THAT THIS DEVID TRANSVENOUS DEFIBRILLATION LEAD EXHIBITED LESS THAN OHMS OF SHOCKING IMPEDANCE THE LEAD AND IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD WERE REMOVED FROM SERVICE

[9]
 MFGID RECEIVED INFORMATION THAT THIS IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD EXHIBITED A CHARGE TIME SECONDS AND GAS GUAGE AT ERI BUT A MONITORING VOLTAGE OF  THE DEVID LEAD ASSOCIATED WITH THE DEVICE WAS REMOVED FROM SERVICE DUE TO A FRACTURE MANIFESTED BY HIGH IMPEDANCE MEASUREMENTS AND SENSING PROBLEMS

[10]
 MFGID RECEIVED INFORMATION THAT THE IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD AND LEAD SYSTEM WAS OVERSENSING POSTSHOCK SEVERAL DAYS AFTER IMPLANT THE OVERSENSING RESULTED IN INAPPROPRIATE SHOCKS AND NONSUSTAINED EPISODES IT WAS FURTHER REPORTED THAT ONE EPISODE OF UNDERSENSING WAS OBSERVED

[20]
 UNWANTEDUNEXPECTED SHOCK INTERFERENCENOISE INAPPROPRIATE THERAPY TESTED OUT OF SPECIFICATIONS DURING MFGRS ANALYSIS

[21]
APPARENT LEAD FRACTURE  PATIENT REPORTS RECEIV

In [416]:
# remove stopwords
def cleanTextRound6(text):
    text = re.sub('''
                    I | ME | MY | MYSELF | WE | OUR | OURS | OURSELVES | YOU | YOUR | YOURS | YOURSELF 
                    | YOURSELVES | HE | HIM | HIS | HIMSELF | SHE | HER | HERS | HERSELF | IT | ITS | ITSELF 
                    | THEY | THEM | THEIR | THEIRS | THEMSELVES | WHAT | WHICH | WHO | WHOM | THIS | THAT | THESE 
                    | THOSE | AM | IS | ARE | WAS | WERE | BE | BEEN | BEING | HAVE | HAS | HAD | HAVING | DO 
                    | DOES | DID | DOING | A | AN | THE | AND | BUT | IF | OR | BECAUSE | AS | UNTIL | WHILE 
                    | OF | AT | BY | FOR | WITH | ABOUT | AGAINST | BETWEEN | INTO | THROUGH | DURING | BEFORE 
                    | AFTER | ABOVE | BELOW | TO | FROM | UP | DOWN | IN | OUT | ON | OFF | OVER | UNDER | AGAIN 
                    | FURTHER | THEN | ONCE | HERE | THERE | WHEN | WHERE | WHY | HOW | ALL | ANY | BOTH | EACH 
                    | FEW | MORE | MOST | OTHER | SOME | SUCH | NO | NOR | NOT | ONLY | OWN | SAME | SO | THAN 
                    | TOO | VERY | S | T | CAN | WILL | JUST | DON | SHOULD | NOW
                    ''', " ", text)
    return text

round6 = lambda x: cleanTextRound6(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round6))
data_clean.head(5)

,FOI_TEXT
0,MFGID RECEIVED INFORMATION THIS IMPLANTABLE C...
1,MFGID RECEIVED INFORMATION THIS PATIENT AN DE...
2,MFGID CRM RECEIVED INFORMATION THIS IMPLANTAB...
3,MFGID MFGID RECEIVED INFORMATION DUE MICROPER...
4,MFGID RECEIVED INFORMATION THIS DEVID TRANSVE...


In [417]:
# indexes 8, 9, 10, 20, 21 were selected to see if cleanTextRound5 worked
for n in [8, 9, 10, 20, 21]:
    print("[" + str(n) + "]\n" + data_clean["FOI_TEXT"][n] + "\n")
# it was successful

[8]
 MFGID RECEIVED INFORMATION THIS DEVID TRANSVENOUS DEFIBRILLATION LEAD EXHIBITED LESS THAN OHMS SHOCKING IMPEDANCE LEAD IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD REMOVED SERVICE

[9]
 MFGID RECEIVED INFORMATION THIS IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD EXHIBITED CHARGE TIME SECONDS GAS GUAGE ERI A MONITORING VOLTAGE  DEVID LEAD ASSOCIATED THE DEVICE REMOVED SERVICE DUE A FRACTURE MANIFESTED HIGH IMPEDANCE MEASUREMENTS SENSING PROBLEMS

[10]
 MFGID RECEIVED INFORMATION THE IMPLANTABLE CARDIOVERTER DEFIBRILLATOR ICD LEAD SYSTEM OVERSENSING POSTSHOCK SEVERAL DAYS IMPLANT OVERSENSING RESULTED INAPPROPRIATE SHOCKS NONSUSTAINED EPISODES WAS REPORTED ONE EPISODE UNDERSENSING OBSERVED

[20]
 UNWANTEDUNEXPECTED SHOCK INTERFERENCENOISE INAPPROPRIATE THERAPY TESTED OF SPECIFICATIONS MFGRS ANALYSIS

[21]
APPARENT LEAD FRACTURE  PATIENT REPORTS RECEIVING SHOCKS HIS LEAD WENT BAD SHORTED OUT



In [418]:
# join the cleaned data with the main df
df["CLEAN_FOI_TEXT"] = data_clean
df["CLEAN_FOI_TEXT"].head()

0     MFGID RECEIVED INFORMATION THIS IMPLANTABLE C...
1     MFGID RECEIVED INFORMATION THIS PATIENT AN DE...
2     MFGID CRM RECEIVED INFORMATION THIS IMPLANTAB...
3     MFGID MFGID RECEIVED INFORMATION DUE MICROPER...
4     MFGID RECEIVED INFORMATION THIS DEVID TRANSVE...
Name: CLEAN_FOI_TEXT, dtype: object

In [419]:
# stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def cleanTextRound7(text): 
    return " ".join([stemmer.stem(word) for word in text.split()])
df["CLEAN_FOI_TEXT"] = df["CLEAN_FOI_TEXT"].apply(cleanTextRound7)


In [420]:
df["CLEAN_FOI_TEXT"]

0        mfgid receiv inform thi implant cardiovert def...
1        mfgid receiv inform thi patient an devid trans...
2        mfgid crm receiv inform thi implant lead float...
3        mfgid mfgid receiv inform due microperfor micr...
4        mfgid receiv inform thi devid transven defibri...
                               ...                        
42203    it report dure implant high pace lead imped ob...
42204    it report the patient experienc phrenic nerv s...
42205    it report the patient present receiv inappropr...
42206    mfgid receiv inform thi patient thi right vent...
42207    mfgid receiv inform at devic chang thi new dev...
Name: CLEAN_FOI_TEXT, Length: 42208, dtype: object

In [421]:
df.to_pickle("./3_COMB_File_CLEANED_1.pkl")